In [1]:
#imports
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample
import os
from os import listdir, path

In [2]:
paths = [
    "Resultados",
    "Data"
]
def crearPaths(paths=paths):
    for p in paths:    
        if not path.exists(f"{os.getcwd()}/{p}"):
            os.mkdir(f"{os.getcwd()}/{p}")
crearPaths()

In [3]:
#correr experimentos (ver como hay que modificar con lo de usar python y c++ juntos)
def correr_experimento(archivo_training, archivo_testing, archivo_salida, k, metodo, nitter, epsilon, alfa):
        
    # Crear proceso para ejecutar el codigo.
    if metodo == '0':
        process = subprocess.Popen(["./build/tp2", archivo_training, archivo_testing, archivo_salida, k, metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    else:
        process = subprocess.Popen(["./build/tp2", archivo_training, archivo_testing, archivo_salida, k, metodo, nitter, epsilon, alfa], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar y leer salida de STDERR con el tiempo de ejecución.
    stdout, stderr = process.communicate() # communicate() devuelve una tupla (stdout, stderr)

    tiempo_de_ejecucion = str.splitlines(stderr)[0]
    #print(clog)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion con el metodo {'kNN' if metodo == 0 else 'PCA'} con los parámetros {archivo_training}, {archivo_testing}, k:{k} {', nitter:'+nitter if metodo == 1 else ''} {', epsilon:'+epsilon if metodo == 1 else ''} {', alfa:'+alfa if metodo == 1 else ''}.")
    return float(tiempo_de_ejecucion)


In [4]:
class Experimento:
    def __init__(self, trainingDataset, testingDataset, outputFile, metodo, vecinos, num_iter, alfa, epsilon):
        self.trainingDataset = trainingDataset
        self.testingDataset = testingDataset
        self.outputFile = outputFile
        self.metodo = metodo
        self.vecinos = vecinos
        self.num_iter = num_iter
        self.alfa = alfa
        self.epsilon = epsilon

    def display(self):
        display(f"Archivo De Entrenamiento: {self.trainingDataset}")
        display(f"Archivo De Prueba: {self.testingDataset}")
        display(f"Archivo De Salida: {self.outputFile}")
        display(f"Metodo: {self.metodo}")
        display(f"Vecinos: {self.vecinos}")
        display(f"Numero De Iteraciones: {self.num_iter}")
        display(f"Alfa: {self.alfa}")
        display(f"Epsilon: {self.epsilon}")

In [5]:
def reduceDataset(n):
    dataset = pd.read_csv("train.csv")
    reducedDataset = resample(dataset, replace = False, n_samples = n)
    return reducedDataset


def splitDataset(dataset, testSize=0.3): #default 70-30 como el fernet
    train, test = train_test_split(dataset, test_size=testSize);
    train.to_csv(path_or_buf="Data/random_train.csv", index=False)
    test.to_csv(path_or_buf="Data/random_validate.csv", index=False)
    return[train, test]


#obsoleta
def k_foldDataset(dataset, k):
    kf = KFold(n_splits=k, shuffle=True)
    fold_indices = kf.split(X=dataset)
    for key, indices in enumerate(fold_indices, 1):
        train_indices = indices[0]
        #test_indices = indices[1]
        fold_train_dataset = dataset.filter(items=train_indices, axis=0) 
        #fold_test_dataset = dataset.filter(items=test_indices, axis=0) 
        fold_train_dataset.to_csv(path_or_buf=f"Data/{k}_fold-{key}.csv", index=False)
        #fold_test_dataset.to_csv(path_or_buf=f"Data/{k}_fold-{key}-test.csv", index=False)
    

In [6]:
#si se corre dos veces sobre la misma cantidad de folds pisa los archivos
def prepareDataSet(size, folds):
    reducedDataset = reduceDataset(size);
    kf = KFold(n_splits=folds, shuffle=True)
    fold_splits = kf.split(X=reducedDataset)

    for key, split in enumerate(fold_splits, 1):
        training_indices = split[0]
        validate_indices = split[1]

        trainDataset = reducedDataset.iloc[training_indices]
        validateDataset = reducedDataset.iloc[validate_indices]
        
        trainDataset.to_csv(path_or_buf=f"Data/{folds}_fold-SET-{key}-train.csv", index=False)
        validateDataset.to_csv(path_or_buf=f"Data/{folds}_fold-SET-{key}-validate.csv", index=False)
                

In [7]:
#Generar experimentos para busqueda de mejor k de knn sin pca en un 5-fold del dataset reducido
def generarExperimentosKnnVariandoK(folds, maxK, step):
    knnExp = []
    for key in range(1,folds+1):
        knnExp += [ Experimento(
            f"./Data/{folds}_fold-SET-{key}-train.csv",
            f"./Data/{folds}_fold-SET-{key}-validate.csv",
            f"./resultados/{folds}_fold-SET-{key}-{k}-vecinos-resultados.csv",
            "0",
            f"{k}",
            None,
            None,
            None
        ) for k in range(1, maxK+1, step)]
    return knnExp
    
def generarExperimentosPCAVariandoK(folds, maxK, step, num_iter, alfa, epsilon):
    knnExp = []
    for key in range(1,folds+1):
        knnExp += [ Experimento(
            f"./Data/{folds}_fold-SET-{key}-train.csv",
            f"./Data/{folds}_fold-SET-{key}-validate.csv",
            f"./resultados/{folds}_fold-SET-{key}-{k}-vecinos-resultados.csv",
            "1",
            f"{k}",
            f"{num_iter}",
            f"{alfa}",
            f"{epsilon}"
        ) for k in range(1, maxK+1, step)]
    return knnExp

def generarExperimentosPCAVariandoNumIter(folds, k, step, maxNum_iter, alfa, epsilon):
    knnExp = []
    for key in range(1,folds+1):
        knnExp += [ Experimento(
            f"./Data/{folds}_fold-SET-{key}-train.csv",
            f"./Data/{folds}_fold-SET-{key}-validate.csv",
            f"./resultados/{folds}_fold-SET-{key}-{k}-vecinos-resultados.csv",
            "1",
            f"{k}",
            f"{num_iter}",
            f"{alfa}",
            f"{epsilon}"
        ) for num_iter in range(1, maxNum_iter+1, step)]
    return knnExp

def generarExperimentosPCAVariandoAlfa(folds, k, step, num_iter, maxAlfa, epsilon):
    knnExp = []
    for key in range(1,folds+1):
        knnExp += [ Experimento(
            f"./Data/{folds}_fold-SET-{key}-train.csv",
            f"./Data/{folds}_fold-SET-{key}-validate.csv",
            f"./resultados/{folds}_fold-SET-{key}-{k}-vecinos-resultados.csv",
            "1",
            f"{k}",
            f"{num_iter}",
            f"{alfa}",
            f"{epsilon}"
        ) for alfa in range(1, maxAlfa+1, step)]
    return knnExp

In [8]:
def extractLabelsFrom(archivo):
    dataset = pd.read_csv(archivo);
    labels = dataset[dataset.columns[0]]
    return labels


def calculateAccuracy(sourceFile, resultsFile):
    expectedLabels = extractLabelsFrom(sourceFile)
    resultLabels = extractLabelsFrom(resultsFile)
    
    hits = 0
    for index, result in enumerate(resultLabels):
        if result == expectedLabels[index]:
            hits += 1
    accuracy = hits/len(expectedLabels)
    return accuracy

def kConMejorAccuracy(metodo, folds, n_from, n_to, step=1):
    df_resultado = pd.read_csv("./resultados/resultado.csv")

    maxAccuracy = 0
    maxAccuracyK = 1
    for i in range(n_from, n_to+1, step):
        df = df_resultado[(df_resultado["Método"] == metodo) & (df_resultado["Vecinos"] == i) & (str(df_resultado["SetEntrenamiento"]).find(str(folds), 0, 3))]
        
        accuracy = np.median(df["Accuracy"])
        print(f"{i}: " + str(accuracy))

        if accuracy > maxAccuracy:
            maxAccuracy = accuracy
            maxAccuracyK = i

    print("Mejor k: " + str(maxAccuracyK) + "\ncon un valor de: " + str(maxAccuracy))
    return maxAccuracyK

def precisionAndRecallMulticase(sourceFile, resultsFile):
    expectedLabels = extractLabelsFrom(sourceFile)
    resultLabels = extractLabelsFrom(resultsFile)

    truePositives = np.array([0,0,0,0,0,0,0,0,0,0])
    falsePositives = np.array([0,0,0,0,0,0,0,0,0,0])
    falseNegatives = np.array([0,0,0,0,0,0,0,0,0,0])
    for index, result in enumerate(resultLabels):
        if result == expectedLabels[index]:
            truePositives[int(result)] += 1
        elif result != expectedLabels[index]:
            falsePositives[int(result)] += 1
            falseNegatives[int(expectedLabels[index])] += 1

    precisions = np.around(np.divide(truePositives, (truePositives + falsePositives)), 4)
    recalls = np.around(np.divide(truePositives, (truePositives + falseNegatives)), 4)

    return [precisions, recalls]




In [9]:
def correr_experimentos(experimentos):
    if not path.exists("./resultados/resultado.csv"):
        columnas=["SetEntrenamiento", "SetTesting", "Resultados", "Método", "Vecinos", "Numero de iteraciones", "Alfa", "Epsilon", "Tiempo", "Accuracy", "p0", "p1", "p2", "p3", "p4", "p5", "p6", "p7","p8", "p9", "r0", "r1", "r2", "r3", "r4", "r5", "r6", "r7","r8", "r9"]
        df_resultado = pd.DataFrame([], columns=columnas);
    else:
        df_resultado = pd.read_csv("./resultados/resultado.csv")

    iteraciones = 5 #PONER EN 5 PARA LA CORRIDA OFICIAL
    for index, experimento in enumerate(experimentos):
        clear_output(wait=True)
        display(f"Experimento: {index+1} de {len(experimentos)} info:")
        experimento.display()
        
        tiempos = []
        for i in range(0, iteraciones):
            tiempo = correr_experimento(
                experimento.trainingDataset, 
                experimento.testingDataset, 
                experimento.outputFile, 
                experimento.vecinos, 
                experimento.metodo, 
                experimento.num_iter, 
                experimento.epsilon, 
                experimento.alfa
            )
            tiempos.append(tiempo)
                
        tiempo_promedio = np.median(tiempos)

        accuracy = calculateAccuracy(experimento.testingDataset, experimento.outputFile)
        [precisions, recalls] = precisionAndRecallMulticase(experimento.testingDataset, experimento.outputFile)
        os.remove(experimento.outputFile) #Una vez calculadas las métricas no nos sirven creo, y asi ahorramos espacio

        if experimento.metodo == "0":
            idList = df_resultado.index[(df_resultado["SetEntrenamiento"] == experimento.trainingDataset)
                                        & (df_resultado["SetTesting"] == experimento.testingDataset) 
                                        & (df_resultado["Resultados"] == experimento.outputFile)
                                        & (df_resultado["Método"] == int(experimento.metodo))
                                        & (df_resultado["Vecinos"] == int(experimento.vecinos))
                                        ].tolist()
        else:
            idList = df_resultado.index[(df_resultado["SetEntrenamiento"] == experimento.trainingDataset)
                                        & (df_resultado["SetTesting"] == experimento.testingDataset) 
                                        & (df_resultado["Resultados"] == experimento.outputFile)
                                        & (df_resultado["Método"] == int(experimento.metodo))
                                        & (df_resultado["Vecinos"] == int(experimento.vecinos))
                                        & (df_resultado["Numero de iteraciones"] == experimento.num_iter)
                                        & (df_resultado["Alfa"] == experimento.alfa)
                                        & (df_resultado["Epsilon"] == experimento.epsilon)
                                        ].tolist()
            

        if len(idList) > 1:
            print("ERROR: Falta filtrar mejor la fila o hay experimentos repetidos")
            df_resultado.to_csv("./resultados/resultado.csv", index=False, header=True);
            break
        elif len(idList) == 0:
            df_resultado = pd.concat([df_resultado, pd.DataFrame([[experimento.trainingDataset, experimento.testingDataset, experimento.outputFile, experimento.metodo, experimento.vecinos, experimento.num_iter, experimento.alfa, experimento.epsilon] + [None for j in range(22)]], columns=df_resultado.columns)])
            if experimento.metodo == "0":
                idList = df_resultado.index[(df_resultado["SetEntrenamiento"] == experimento.trainingDataset)
                                            & (df_resultado["SetTesting"] == experimento.testingDataset) 
                                            & (df_resultado["Resultados"] == experimento.outputFile)
                                            & (df_resultado["Método"] == experimento.metodo)
                                            & (df_resultado["Vecinos"] == experimento.vecinos)
                                            ].to_list()
            else:
                idList = df_resultado.index[(df_resultado["SetEntrenamiento"] == experimento.trainingDataset)
                                            & (df_resultado["SetTesting"] == experimento.testingDataset) 
                                            & (df_resultado["Resultados"] == experimento.outputFile)
                                            & (df_resultado["Método"] == int(experimento.metodo))
                                            & (df_resultado["Vecinos"] == int(experimento.vecinos))
                                            & (df_resultado["Numero de iteraciones"] == experimento.num_iter)
                                            & (df_resultado["Alfa"] == experimento.alfa)
                                            & (df_resultado["Epsilon"] == experimento.epsilon)
                                            ].tolist()

        id = idList[0]
        
        df_resultado.loc[id, 'Tiempo'] = tiempo_promedio
        df_resultado.loc[id, 'Accuracy'] = accuracy
        for j in range(10):
            df_resultado.loc[id, f'p{j}'] = precisions[j]
            df_resultado.loc[id, f'r{j}'] = recalls[j]

    df_resultado.to_csv("./resultados/resultado.csv", index=False, header=True);



In [11]:
folds = 5
experimentos = []

# prepareDataSet(10000, folds)
# experimentos = generarExperimentosKnnVariandoK(folds, 2001, 50)
# correr_experimentos(experimentos)

# kConMejorAccuracy(0, 5, 1, 2001, 50)

# prepareDataSet(42000, folds)
# experimentos = generarExperimentosKnnVariandoK(folds, 51, 5)
# correr_experimentos(experimentos)

# kConMejorAccuracy(0, 5, 1, 51, 5)

# experimentos = generarExperimentosKnnVariandoK(folds, 5, 1)
# correr_experimentos(experimentos)

# kConMejorAccuracy(0, 5, 1, 5)

experimentos = generarExperimentosPCAVariandoK(folds, 51, 5, 1000, 20, 0.000001)
correr_experimentos(experimentos)

kConMejorAccuracy(1, 5, 1, 51, 5)

'Experimento: 1 de 55 info:'

'Archivo De Entrenamiento: ./Data/5_fold-SET-1-train.csv'

'Archivo De Prueba: ./Data/5_fold-SET-1-validate.csv'

'Archivo De Salida: ./resultados/5_fold-SET-1-1-vecinos-resultados.csv'

'Metodo: 1'

'Vecinos: 1'

'Numero De Iteraciones: 1000'

'Alfa: 20'

'Epsilon: 1e-06'

IndexError: list index out of range

In [17]:
kConMejorAccuracy(metodo = 0, folds = 5, n_from = 1, n_to = 51, step = 5)  

print(1e-6)

1: 0.9217857142857144
6: 0.911547619047619
11: 0.911547619047619
16: 0.911547619047619
21: 0.911547619047619
26: 0.911547619047619
31: 0.911547619047619
36: 0.911547619047619
41: 0.911547619047619
46: 0.911547619047619
51: 0.9092857142857144
Mejor k: 1
con un valor de: 0.9217857142857144
1e-06
